# Champion Classifier:
- match image data with excel file of champion labels
- background images
- VGG structure

- annotations_file (CSV)
    - file_name, one_hot_encoding_champion, champion_name 

- img_dir (Path)
    - path that contains all the image


In [26]:
# Packages
import numpy as np
import pandas as pd
import os
import shutil #copy and paste files
import re
import random

### Read data

In [2]:
# Data of cropped images
df = pd.read_excel('data/Champions_Label.xlsx')
# Data of all champion names
master_champ_df = pd.read_excel('data/champion_list_master_dataset.xlsx')

# Columns to keep
columns = ['youtuber', 'video_name', 'frame_name', 'cropped_name', 'champion_name']

### Clean data

In [3]:
# list of all unique champions + background
master_champ_list = master_champ_df.champion_name.unique().tolist() + ['Background']

In [25]:
# Remove all NA values
df = df[~df.champion_name.isna()]

# Keep relevant columns
df = df.loc[:, columns]

# Check if names are all correct
df[~df.champion_name.isin(master_champ_list)]

,youtuber,video_name,frame_name,cropped_name,champion_name


In [27]:
# One hot encoding of champions
champion_df = df.champion_name.copy()
champion_df = champion_df.apply(lambda x: master_champ_list.index(x))

df['champion_onehot'] = champion_df

### Move all relevant images to data

In [61]:
def move_images(final_data_path, df):

    #final_data_path = os.path.join(os.getcwd(), 'data', 'champion-classifier')

    if not os.path.exists(final_data_path):
        os.makedirs(final_data_path)

    image_name_list = []
    image_path_list = []

    for i in range(len(df)):
        image_name = f'image_{i}.jpg'
        row = df.iloc[i, :]
        path_to_img = os.path.join(os.getcwd(), 'contents', row.youtuber, row.video_name, re.sub(r'.jpg', '', row.frame_name), re.sub(r'.jpg', '', row.cropped_name)+'.png')
        path_to_final_img = os.path.join(final_data_path, image_name)
        image_name_list.append(image_name)
        image_path_list.append(path_to_final_img)
        shutil.copyfile(path_to_img, path_to_final_img)

    df['image_name'] = image_name_list
    df['image_file_path'] = image_path_list


    return df

move_images(final_data_path = os.path.join(os.getcwd(), 'data', 'champion-classifier'), df = df)

In [33]:
annotation_file = df.loc[:, ['image_name', 'champion_onehot', 'champion_name']]

annotation_file = annotation_file.reset_index(drop=True)

annotation_file.to_csv(os.path.join(os.getcwd(), 'data', 'annotation_file.csv'))

annotation_file.sample(10)

,image_name,champion_onehot,champion_name
35,image_35.jpg,4,Ezreal
832,image_832.jpg,46,Orianna
506,image_506.jpg,30,Leona
819,image_819.jpg,39,Ahri
473,image_473.jpg,31,Lucian
374,image_374.jpg,68,Tristana
253,image_253.jpg,79,Sion
25,image_25.jpg,63,Twisted Fate
282,image_282.jpg,83,Yuumi
770,image_770.jpg,35,Senna


# Create batch images (different from original training data)

In [62]:
# Data of cropped images
df = pd.read_excel('data/Champions_Label.xlsx')
# Data of all champion names
master_champ_df = pd.read_excel('data/champion_list_master_dataset.xlsx')

# Columns to keep
columns = ['youtuber', 'video_name', 'frame_name', 'cropped_name', 'champion_name']

# list of all unique champions + background
master_champ_list = master_champ_df.champion_name.unique().tolist() + ['Background']

In [63]:
df_unnamed = df[df.champion_name.isna()].reset_index(drop=True)

In [64]:
# DO NOT CHANGE THE ORDER OF THIS LIST:
video_list_check = ['100 HP Challenger - Nerfed but still so good  TFT Gizmos _ Gadgets  Teamfight Tactics.mp4']
                        # '11 Loss Mercenary - 29 HP to 55 HP!  TFT Gizmos _ Gadgets  Teamfight Tactics.mp4',
                        # '3 Mutant Chogath - Augments are crazy!!  TFT Gizmos _ Gadgets  Teamfight Tactics.mp4',
                        # '4 Score Ahri - Gamble with Confidence  TFT Neon Nights  Teamfight Tactics.mp4',]

for i in range(len(video_list_check)):
    df_unamed_sample = df_unnamed[df_unnamed.video_name == video_list_check[i]]
    final_data_path = os.path.join(os.getcwd(), 'data', f'batch-data-{i}')

    move_images(final_data_path, df_unamed_sample)


/var/folders/f7/7wm8ypsx1_zcy_df6_f5nrl40000gn/T/ipykernel_4950/986051482.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_name'] = image_name_list
/var/folders/f7/7wm8ypsx1_zcy_df6_f5nrl40000gn/T/ipykernel_4950/986051482.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_file_path'] = image_path_list


In [75]:
# Overwrite master spreadsheet to include image_file_pathpath

df.merge(df_unamed_sample, how='left').to_excel(os.path.join('data', 'Champions_Label.xlsx'))